<a href="https://colab.research.google.com/github/salsaimon/proyect_analitica_win/blob/main/preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:

import sys
import os

path='/content/drive/MyDrive/analitica3win/proyect_analitica_win'

os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

In [13]:
con=sql.connect('base.db')  ### crea base de datos
cur=con.cursor()  ### para ejecutar consultas en la base de datos

In [14]:

#### Cargar paquetes siempre al inicio
import pandas as pd ### para manejo de datos
import sqlite3 as sql #### para bases de datos sql
import a_funciones as funciones  ###archivo de funciones propias
import sys ## saber ruta de la que carga paquetes

In [9]:
####################################################################################################################
########################  1. Comprender y limpiar datos ##################################################################
####################################################################################################################
########   Verificar lectura correcta de los datos
########   Verificar Datos faltantes (eliminar variables si es necesario) (la imputación está la parte de modelado)
########   Tipos de variables (categoricas/numéricas/fechas)
########   Niveles en categorícas
########   Observaciones por categoría
########   Datos atípicos en numéricas

In [28]:
##leer bases de datos

retiros=pd.read_csv('data/retirement_info.csv')
general=pd.read_csv('data/general_data.csv')
employee_survey=pd.read_csv('data/employee_survey_data.csv')
manager_survey=pd.read_csv('data/manager_survey.csv')

#crear df para cada base
df_retiros= retiros
df_general= general
df_employee= employee_survey
df_manager= manager_survey

###### Verificar lectura correcta de los datos
df_retiros.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_general.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_employee.sort_values(by=['EmployeeID'],ascending=1).head(100)
df_manager.sort_values(by=['EmployeeID'],ascending=1).head(100)


,EmployeeID,JobInvolvement,PerformanceRating,SurveyDate
0,1,3,3,2015-12-31
4410,1,3,3,2016-12-31
1,2,2,4,2015-12-31
2,3,3,3,2015-12-31
4412,3,3,3,2016-12-31
...,...,...,...,...
4461,52,2,3,2016-12-31
51,52,2,3,2015-12-31
4462,53,3,4,2016-12-31
52,53,3,4,2015-12-31


In [39]:

##### resumen con información tablas faltantes y tipos de variables y hacer correcciones
df_retiros.info(verbose=True)
df_general.info(verbose=True)
df_employee.info(verbose=True)
df_manager.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   EmployeeID         711 non-null    int64         
 1   Attrition          711 non-null    object        
 2   retirementDate     711 non-null    datetime64[ns]
 3   retirementType     711 non-null    object        
 4   resignationReason  641 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 27.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Age                      8820 non-null   int64         
 1   BusinessTravel           8820 non-null   object        
 2   Department               8820 non-null   object        
 3   DistanceFromHome        

In [36]:
#### Convertir campos a formato fecha
df_retiros["retirementDate"]=pd.to_datetime(df_retiros['retirementDate'], format="%d/%m/%Y")
df_general["InfoDate"]=pd.to_datetime(df_general['InfoDate'], format="%d/%m/%Y")
df_employee["DateSurvey"]=pd.to_datetime(df_employee['DateSurvey'], format="%d/%m/%Y")
df_manager["SurveyDate"]=pd.to_datetime(df_manager['SurveyDate'], format="%d/%m/%Y")

In [38]:
#### convertir a categórica

#general
df_general=df_general.astype({'Education': object,'JobLevel': object,'StockOptionLevel': object})
#employee
df_employee=df_employee.astype({'EnvironmentSatisfaction': object,'JobSatisfaction': object,'WorkLifeBalance': object})
#manager
df_manager=df_manager.astype({'JobInvolvement': object,'PerformanceRating': object})

In [ ]:
###Eliminar columnas que no se utilicen
df_employees=df_employees.drop(["PayRate","MgrID","RaceID","TermDt"], axis=1) # PayRate no tiene datos, MgrID no se va a usar es el código del jefe, RaceID se va a usar la descripción


In [26]:
print(df_retiros)


     EmployeeID Attrition retirementDate retirementType resignationReason
0             2       Yes     2016-10-15    Resignation            Others
1             7       Yes     2016-07-20    Resignation            Stress
2            14       Yes     2016-11-08    Resignation            Others
3            29       Yes     2016-10-15    Resignation            Others
4            31       Yes     2016-02-08    Resignation            Salary
..          ...       ...            ...            ...               ...
706        4382       Yes     2015-04-18    Resignation            Salary
707        4387       Yes     2015-09-16    Resignation            Others
708        4389       Yes     2015-05-24    Resignation            Others
709        4392       Yes     2015-02-26          Fired               NaN
710        4403       Yes     2015-01-26    Resignation            Stress

[711 rows x 5 columns]


In [ ]:
retiros.to_sql('retiros',con,index=False, if_exists='replace') ### para llevar un data frame de pandas a SQL
general.to_sql('general',con,index=False, if_exists='replace')
employee_survey.to_sql('employee_survey',con,index=False, if_exists='replace')
manager_survey.to_sql('manager_survey',con,index=False, if_exists='replace')

In [ ]:
df_retiros=pd.read_csv(retiros)